---
# 신경망 종합 테스트
- burst 하여 active cell 이 많으면 새로운 패턴을 인지한다는 의미
- 먼저 spatial pooler 가 안정되어야 한다.
---
encoder - sp - tm -> prediction + textual representation<br/>
예측 cell / 활성화 cell 비교 - 예측이 맞는지 확인 가능<br/>
예측 column / 입력 column 비교 - 마찬가지<br/>
burst 한 column / 활성화 된 column - 마찬가지<br/>


In [1]:
import numpy as np
import random
import nbimporter
import Encoder
import SP3
import TM

Importing Jupyter notebook from Encoder.ipynb
Importing Jupyter notebook from SP3.ipynb
Importing Jupyter notebook from TM.ipynb
Importing Jupyter notebook from memoryStruct.ipynb


In [2]:
encoder = Encoder.ScalarEncoder(out_size=100, w=20, max_val=30)
sp = SP3.SpatialPooler(encoder.out_size, columns=512, perm_conn=.6, minOver=encoder.w/5, potential_rate=0.9)
tm = TM.TemporalMemory(sdr_size=sp.columnCount)

In [3]:
# for i in range(30):
    a = np.arange(encoder.max_val)
    random.shuffle(a)
    
    for i in a:
        input_data = encoder.encode(i)
        sp.compute_SDR(input_data)

In [4]:
for i in range(encoder.max_val):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()
    
    sp.similarity(sdr1, sdr2, False)
    print('')

유사도 : 0.47058823529411764
[157 158 234 249 282 287 296 310 319 321 383 396 408 422 461 473 486]
[ 11  85 117 177 199 249 295 296 302 307 310 319 321 375 422 454 473 477
 486]

유사도 : 0.3684210526315789
[ 11  85 117 177 199 249 295 296 302 307 310 319 321 375 422 454 473 477
 486]
[ 10 151 177 178 217 241 295 307 310 319 351 396 406 473 486 504]

유사도 : 0.9375
[ 10 151 177 178 217 241 295 307 310 319 351 396 406 473 486 504]
[  5  10  27  33  44  49  58  72 106 141 148 177 178 199 210 217 241 250
 262 278 281 287 295 304 307 309 310 319 326 351 396 406 408 420 422 461
 473 486 501 504 506]

유사도 : 0.2682926829268293
[  5  10  27  33  44  49  58  72 106 141 148 177 178 199 210 217 241 250
 262 278 281 287 295 304 307 309 310 319 326 351 396 406 408 420 422 461
 473 486 501 504 506]
[ 27  44 106 116 177 199 210 220 250 268 278 281 351 486]

유사도 : 0.5
[ 27  44 106 116 177 199 210 220 250 268 278 281 351 486]
[ 27 116 148 177 195 199 250 251 300 350 351 462 469 486]

유사도 : 0.5
[ 27 116 148 177

In [4]:
seq_data = [1,2,3]

for i in range(1):
    for s in seq_data:
        dat = encoder.encode(i)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()
        tm.compute(sdr)

In [18]:
# len(tm.connInfo.totalSegments)
cell = 294
print('{} 번 cell 에 연결되어 있는 synapse : {}'.format(cell, tm.connInfo.totalCells[cell].connectedSynapses))
#print(len(tm.prev_activeCells))
#print(len(tm.activeCells))
#for c in tm.connInfo.totalCells:
 #   if(len(c.connectedSynapses) > 0):
  #      print("살아있는 cell : " + str(c.cell_id))
        
#choose = list(tm.connInfo.totalCells[cell].connectedSynapses)[0]
#for i in tm.connInfo.totalSegments:
 #   for j in i.synapses:
  #      if (j.synapse_id == choose):
   #         print(j.synapse_id)
    #        print(j.preSynapticCell)
     #       print(j.permanence)
        
print(sp.getActiveColumns())
print(tm.predicted)
print(tm.bursted)

294 번 cell 에 연결되어 있는 synapse : set()
[ 22  38  43  46 104 132 162 287 348 350 360 416 461]
[348, 360, 416, 461, 22, 38, 43, 104, 132, 162, 287, 348, 350, 22, 43, 46, 104, 162, 287]
[22, 43, 46, 104, 162, 287, 348, 360, 416, 461]
